# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Note: pandas and numpy are already imported as pd and np
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Please print your final result or dataframe

# Ensure 'visit_date' is in correct datetime format
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

# Filter visit_date for July 2024
july_dates = fct_guest_spending[
  (fct_guest_spending['visit_date'].dt.month == 7) &
  (fct_guest_spending['visit_date'].dt.year == 2024)
]

# Get average amount spent per park experience type for July 2024
avg_spend = july_dates.groupby('park_experience_type')['amount_spent'].mean().reset_index()

# Get full list of all park experience types
all_types = fct_guest_spending[['park_experience_type']].drop_duplicates()

# Merge to ensure all park experience types appear, even with no July transactions
merged = all_types.merge(avg_spend, on='park_experience_type', how='left')

# Fill missing averages with 0.0
merged['amount_spent'] = merged['amount_spent'].fillna(0.0)

# Print the final result
for _, row in merged.iterrows():
    print(f"{row['park_experience_type']}: {row['amount_spent']}")

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
# Ensure 'visit_date' is datetime
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

# Filter for visits in August 2024
aug_dates = fct_guest_spending[
    (fct_guest_spending['visit_date'].dt.month == 8) &
    (fct_guest_spending['visit_date'].dt.year == 2024)
]

# Count visits per guest
visit_counts = aug_dates.groupby('guest_id').size()

# Guests with more than one visit in August
multiple_visitors = visit_counts[visit_counts > 1].index

# Filter data for these guests only
multi_visitors_data = aug_dates[aug_dates['guest_id'].isin(multiple_visitors)]

# Sort by guest_id and visit_date to ensure sequential order
multi_visitors_data = multi_visitors_data.sort_values(['guest_id', 'visit_date'])

# Group by guest_id and get spending on first and last visit
spending = multi_visitors_data.groupby('guest_id')['amount_spent'].agg(['first', 'last']).reset_index()

# Calculate difference: last visit spending minus first visit spending
spending['spending_difference'] = spending['last'] - spending['first']

print(spending)

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
# Filter for sept 2024
sept_dates = fct_guest_spending[
  (fct_guest_spending['visit_date'].dt.month == 9) &
  (fct_guest_spending['visit_date'].dt.year == 2024)
]

# Total spending per guest
# Group by guest_id
grouped = sept_dates.groupby('guest_id')
# Total amount_spent
total_spent = grouped['amount_spent'].agg(sum)
total_spent = total_spent.reset_index()

# Categorisation into Low, Medium, High Spending
total_spent['spending_segment'] = pd.cut(
    total_spent['amount_spent'],
    bins=[0, 50, 100, float('inf')],
    labels=['Low', 'Medium', 'High'],
    right=False  # include lower bound, exclude upper
)

# Exclude guests with no fct_guest_spending
total_spent = total_spent[total_spent['amount_spent'] > 0]

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)